## Introduction

This notebook demonstrates a methodology of identifying avalanche initiation areas, i.e. the method uses as import the scar of each avalanche from a historical record and tries to identify the higher 20% of this scar and define it as the initiation area.

The statistics of elevation of each shape imported to the dataframe (e.g. count, mean min, etc.) were acquired from Zonal Statistics in QGIS from the avalanche polygons to the DEM.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ginamanou/avalanche-susceptibility/blob/main/4.%20Initiation_areas_with_quantiles_approach.ipynb)

## Methodology

In [ ]:
# Import packages 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

import plotly.graph_objects as go
from shapely.geometry import Point

In [ ]:
path = "D:/Allaus/Code/cont10_pez_sin.csv"
df = pd.read_csv(path)
df

In [ ]:
df1 = df.copy()

In [ ]:
df1.shape

In [ ]:
df1.dtypes

In [ ]:
df1.columns

In [ ]:
df1 = df1.drop(columns=['fid', 'x_cont', 'y_cont', 'x', 'y'])
df1

In [ ]:
df1 = df1.rename(columns = {'ELEV': 'elevation', 'CODI_DEL_L': 'code', 'area_calc': 'area', 
                            'Xcode_cont': 'x', 'Ycode_cont': 'y'})
df1

In [ ]:
df1.describe()

In [ ]:
df1.hist(column='elevation')
plt.show()

In [ ]:
code_names = list(df1['code'].unique())   
len(code_names)  

In [ ]:
# see = df1[~df1.code.isin(code_names)]
# see

In [ ]:
quantiles = pd.DataFrame(df1.groupby(['code'])['elevation'].quantile(q=0.80)).reset_index()
quantiles

In [ ]:
quantiles = quantiles.rename(columns = {'elevation': 'quant80'})
quantiles

In [ ]:
ascend = df1.sort_values(by='code', ignore_index=True)
ascend

In [ ]:
tot = pd.merge(ascend, quantiles, how='outer', on='code')
tot

In [ ]:
tot.loc[tot['elevation'] >= tot['quant80'], 'keepornot'] = 1
tot.loc[tot['elevation'] < tot['quant80'], 'keepornot'] = 0
tot

In [ ]:
new = tot.loc[tot['keepornot'] == 1]
new

In [ ]:
print(len(list(new['code'].unique())))     # check

In [ ]:
last = pd.DataFrame(new.groupby(['code'])['elevation'].min()).reset_index()
last

In [ ]:
last.rename(columns={'elevation': 'elevation_min'}, inplace=True)
last

In [ ]:
tot2 = pd.merge(new, last, how='outer', on='code')
tot2

In [ ]:
tot2.loc[tot2['elevation'] == tot2['elevation_min'], 'onezero'] = 1
tot2.loc[tot2['elevation'] != tot2['elevation_min'], 'onezero'] = 0
tot2

In [ ]:
new2 = tot2.loc[tot2['onezero'] == 1]  # length > unique values, because of repeated intersections of contours in one polygon
new2

In [ ]:
print(len(list(new2['code'].unique())))   # good

In [ ]:
duplicateRows = new2[new2.duplicated(['code'])]

In [ ]:
print(duplicateRows)

In [ ]:
# Let's export it and pass it to QGIS

new2.to_csv("D:/Allaus/Code/points_pez_sin.csv")